In [ ]:
"""
Γεμίζω τον πίνακα με τις μετρήσεις
του labevent 8ώρου.

Ο πίνακας μου απαρτίζεται από 388 στήλες και 68.191 γραμμές,
οπότε μου δημιουργούν 26.458.108 κελιά.

Οι τιμές μου ανέρχονται στα 1.276.856 οπότε με μια αφαίρεση
προκείπτουν 25.181.252 NaN κελιά, πράγμα που επιβεβαιώνει
και η εντολή df_lab_tbl.isna().sum().sum()

"""

In [1]:
import pandas as pd

In [2]:
df_lab_tbl = pd.read_csv('D:\Thesis\exports\o17_empty_lab_tbl.csv', index_col = False)
df_data_labevents = pd.read_csv('D:\Thesis\mimic-iv-1.0\hosp\d_labitems.csv.gz')
df_mean_lab = pd.read_csv('D:\Thesis\exports\o16_labevents_grouped_8hours_date_time.csv')

In [3]:
# Φέρνω τις τελευταίες στήλες, πρώτες
for i in range (1, 5):
    cols = list(df_lab_tbl.columns)
    cols = [cols[-1]] + cols[:-1]
    df_lab_tbl = df_lab_tbl[cols]

In [4]:
# Ελέγχω αν υπάρχει NaN στην στήλη με τις μέσες τιμές, το αποτέλεσμα ισούται με μηδέν.
df_mean_lab['Mean_Lab'].isna().sum()

0

In [5]:
"""
Δημιουργώ εκ νέου την στήλη concat χωρίς
κενά ανάμεσα στα στοιχεία. Θα χρησιμοποιήσω
αυτή τη στήλη σαν index και τα κενά μπορεί
να δημιουργήσουν πρόβλημα.
"""
df_lab_tbl["concat"] = (df_lab_tbl["subject_id"].astype(str)
                          + df_lab_tbl["hadm_id"].astype(str)
                          + df_lab_tbl["date"].astype(str)
                          + df_lab_tbl["time"].astype(str))

# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace('-','')
df_lab_tbl['concat'] = df_lab_tbl['concat'].str.replace(':','')

# Ορίζω την στήλη concat ως index των γραμμών
df_lab_tbl = df_lab_tbl.set_index('concat')

In [6]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες.
"""
df_mean_lab["concat"] = (df_mean_lab["subject_id"].astype(str)
                         + df_mean_lab["hadm_id"].astype(str)
                         + df_mean_lab["date"].astype(str)
                         + df_mean_lab["time"].astype(str))

# Αφαιρώ τους χαρακτήρες - και : από της τιμές της concat
df_mean_lab['concat'] = df_mean_lab['concat'].str.replace('-','')
df_mean_lab['concat'] = df_mean_lab['concat'].str.replace(':','')

In [7]:
for row in df_mean_lab.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Mean_Lab
    df_lab_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [8]:
# Έλεγχος NaN περιοχών σε όλο το dataframe
df_lab_tbl.isna().sum().sum()

55801191

In [9]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_lab_tbl.to_csv('D:\Thesis\exports\o18_fill_lab_itemid_header_tbl.csv', index=False)

## Αντικατάσταση των κεφαλίδων

In [10]:
df_data_labevents = df_data_labevents.set_index('itemid')

In [11]:
# Ενοποίηση τιμών σε κελιά
df_data_labevents["header"] = (df_data_labevents["label"].astype(str)
                                                + " ("
                                                + df_data_labevents["fluid"].astype(str)
                                                + ")")

In [12]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικαταστάσεις.
"""
for col in df_lab_tbl.columns:
    if ((col != 'subject_id')
        and (col != 'hadm_id')
        and (col != 'date')
        and (col != 'time')):
        temp = (df_data_labevents.loc[int(col), 'header'])
        df_lab_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [13]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τα label των παρατηρήσεων
df_lab_tbl.to_csv('D:\Thesis\exports\o19_fill_lab_label_header_tbl.csv', index=False)

In [14]:
df_lab_tbl

,subject_id,hadm_id,date,time,Alveolar-arterial Gradient (Blood),Base Excess (Blood),"Calculated Bicarbonate, Whole Blood (Blood)",Calculated Total CO2 (Blood),Carboxyhemoglobin (Blood),"Chloride, Whole Blood (Blood)",...,Plasma (Cerebrospinal Fluid),Polys (Cerebrospinal Fluid),Promyelocytes (Cerebrospinal Fluid),"RBC, CSF (Cerebrospinal Fluid)","Total Nucleated Cells, CSF (Cerebrospinal Fluid)","Total Nucleated Cells, Joint (Joint Fluid)",Lining Cell (Other Body Fluid),"Total Nucleated Cells, Other (Other Body Fluid)","Total Nucleated Cells, Pleural (Pleural)",Absolute Lymphocyte Count (Blood)
concat,,,,,,,,,,,,,,,,,,,,,
100009802691386521890628080000,10000980,26913865,2189-06-28,08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009802691386521890628000000,10000980,26913865,2189-06-28,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009802691386521890629080000,10000980,26913865,2189-06-29,08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009802691386521890630000000,10000980,26913865,2189-06-30,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009802691386521890701000000,10000980,26913865,2189-07-01,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199974732778749421731001160000,19997473,27787494,2173-10-01,16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199974732778749421731002000000,19997473,27787494,2173-10-02,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199974732778749421730916080000,19997473,27787494,2173-09-16,08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
